In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# This is the YouTube video we're going to use.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=F1N1TpnR5mI&ab_channel=Three-EyedRaven"

In [3]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser


In [ ]:
import tempfile
import whisper
from pytube import YouTube


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists("got_transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("got_transcription.txt", "w") as file:
            file.write(transcription)

In [10]:
import re

# Read the transcript from the file
with open('timestamp.txt', 'r') as file:
    transcript = file.read()

# Define a regular expression to match timestamps (e.g., [00:00:05])
timestamp_pattern = r'\[?\d{1,2}:\d{2}(:\d{2})?\]?'

# Remove all timestamps from the transcript
cleaned_transcript = re.sub(timestamp_pattern, '', transcript)

# Optionally, remove extra spaces and newlines left behind after removing timestamps
cleaned_transcript = re.sub(r'\s+', ' ', cleaned_transcript).strip()

# Save the cleaned transcript to a new file
with open('got_transcription.txt', 'w') as file:
    file.write(cleaned_transcript)




In [12]:
import re
import os

# Read the transcript from the file
with open('timestamp.txt', 'r') as file:
    transcript = file.read()

# Define a regular expression to match timestamps (e.g., [00:00:05])
timestamp_pattern = r'\[?\d{1,2}:\d{2}(:\d{2})?\]?'

# Remove all timestamps from the transcript
cleaned_transcript = re.sub(timestamp_pattern, '', transcript)

# Optionally, remove extra spaces and newlines left behind after removing timestamps
cleaned_transcript = re.sub(r'\s+', ' ', cleaned_transcript).strip()

# Save the cleaned transcript to a new file if it's not already present
output_file = 'got_transcription.txt'
if not os.path.exists(output_file):
    with open(output_file, 'w') as file:
        file.write(cleaned_transcript)
else:
    # Check if the cleaned transcript is not already present in the file
    with open(output_file, 'r') as file:
        existing_transcript = file.read()
    if cleaned_transcript not in existing_transcript:
        with open(output_file, 'a') as file:
            file.write('\n' + cleaned_transcript)
